In [3]:
%pip install cryptography

   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   ---------------------------------------- 3.4/3.4 MB 66.7 MB/s eta 0:00:00

   ------------- -------------------------- 1/3 [cffi]
   -------------------------- ------------- 2/3 [cryptography]
   ---------------------------------------- 3/3 [cryptography]

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sys
sys.path.append(r"C:\Users\USER\Desktop\GitHub\3rd_project")  # chahae 폴더의 상위 폴더

from chahae.github_repo_viewer import main


documents = main("dfjoefjjdk")


[설치] python-dotenv 패키지를 설치합니다...
[설치] python-dotenv 패키지 설치 완료

[환경] GitHub 토큰을 입력해주세요.
GitHub 토큰: 

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

def remove_db(db_name):    
    # 컬렉션을 직접 삭제
    Chroma.delete_collection(name="your_collection_name")

def make_chroma_db(documents):
    # Chunking
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = splitter.split_documents(documents)

    # 벡터 저장소 만들기
    db = Chroma.from_documents(docs, OpenAIEmbeddings(), persist_directory="chroma_db")
    return db

def get_top5_docs_from_db(query, db):
    retriever = db.as_retriever(search_kwargs={"k": 5}) # 상위 5개만 추출하도록 설정

    return retriever.get_relevant_documents(query)


In [ ]:
from openai import OpenAI

# OpenAI API 키 설정 (환경변수 또는 직접 입력)
client = OpenAI()

# 🔍 GPT를 사용해 요약 생성
def summarize_with_gpt(content: str, file_path: str, max_chars: int = 1500) -> str:
    prompt = f"""
    다음은 '{file_path}'라는 파일의 코드입니다. 
    이 파일의 목적이 무엇인지, 어떤 기능이 있고 어떤 문제를 해결하는지 간단히 요약해 주세요. 
    \n\n```python\n{content[:max_chars]}\n```\n\n요약:"""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4.1",  # 또는 gpt-3.5-turbo
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ GPT 요약 실패 ({file_path}): {e}")
        return "요약 실패"